
---

# Silhouette masking with OpenCV

---

One component of the art/tech project I'm working on for the Dec 2019 exhibition requires creating silhouette masks of people moving around a webcam. This notebook explores different ways to do that.

---

## Setup

---

In [1]:
import os

import cv2 as cv
import imutils
import numpy as np

from imutils.video import FPS

---

## Detection via background subtraction

---

One possibility for silhouette masking is _background subtraction_ - using motion and color changes in a video feed to find static background regions vs. dynamic foreground regions OpenCV has multiple background subtraction methods.

---

## Webcam

Press 'Q' to quit.

#### Stream resolution

In [2]:
width_stream = 160
height_stream = 90

#### Display resolution

In [3]:
width_display = 960
height_display = 540

# Plus some useful auxiliary stuff
aspect_ratio = height_display / width_display
screen_inverse = (1 / width_display, 1 / height_display)

#### Visualization parameters

In [9]:
# Decay constant. Each frame, the fx field's values get multiplied
# by this
c_decay = 0.996
# Mask emission strength constant.
c_emit = 0.002
# Sorta hard to describe overall intensity constant
c_viz = 20

#### Visualization function utilities

In [10]:
def cos1(x):
    return 0.5 * (np.cos(x) + 1)

def mod1(x):
    return ((x % 1) + 1) % 1

def sigmoid(x, c, m):
    return 1. / (1. + np.exp(-m * (x - c)))

#### Visualization function constants

In [11]:
coords_x = np.linspace(-0.5, 0.5, width_display)
coords_y = np.linspace(-0.5 * aspect_ratio, 0.5 * aspect_ratio, height_display)
grid_x, grid_y = np.meshgrid(coords_x, coords_y)
grid = (grid_x, grid_y)

tt = 20 * (1. - np.cos(0.01745329251 * (c_viz + 15.)))
theta = 6.28318530718 * mod1(0.005 * tt)
ct = np.cos(theta)
st = np.sin(theta)

ur = ct * grid_x + st * grid_y
vr = -st * grid_x + ct * grid_y

dc = 0.75 * cos1(0.021 * tt)

uc = dc + (1 - dc) * ur
vc = dc + (1 - dc) * vr

ut = uc * tt
vt = vc * tt
rt = (uc * uc + vc * vc) * tt

scrt = np.sin(cos1(0.05 * rt))

#### Visualization functions

In [12]:
def visualize(field, mask, count_on):
    """
    
    """
    hmod = 0.33 + (3 + 0.2 * tt) * field
    dh = scrt + 0.3 * np.cos(0.1 * ut * hmod) \
        * np.sin(grid_x + grid_y * tt * hmod + 2. * np.cos(0.27 * ut))
    h = mod1(0.93 + 0.21 * dh)
    
    smod = 1 + np.multiply(field, field)
    s = np.minimum(1, (field + 0.25 * cos1(0.8 * tt)) * smod)
    b = np.ones_like(s)
#     print(h.shape, s.shape, b.shape)
    hsb = (255 * np.stack((h, s, b), axis=-1)).astype(np.uint8)
    bgr = cv.cvtColor(hsb, cv.COLOR_HSV2BGR)
    mask_multiplier = np.minimum(1, 0.5 + 0.25 * mask + 0.05 * count_on)
    return np.multiply(bgr, np.stack([mask_multiplier]*3, axis=-1)).astype(np.uint8)

In [13]:
cap = cv.VideoCapture(1)
cap.set(3, width_stream)
cap.set(4, height_stream)

fgbg = cv.createBackgroundSubtractorKNN(100, 100, True)

field = np.zeros((height_display, width_display), dtype=np.float32)
count_on = np.zeros((height_display, width_display), dtype=np.float32)
# cv.namedWindow('frame', cv.WINDOW_NORMAL)
# cv.namedWindow('mask', cv.WINDOW_NORMAL)

fps = FPS().start()

t = 0

while(1):
    ret, frame = cap.read()
    
    fgmask = 255 * (fgbg.apply(frame) > 0).astype(np.uint8)
    fgmask = cv.medianBlur(fgmask, 11)
    fgmask = cv.GaussianBlur(fgmask, (5, 5), 0)
    fgmask_display = (imutils.resize(fgmask, width=width_display, inter=cv.INTER_CUBIC)).astype(np.float32) / 255
    
    count_on = np.maximum(0, count_on - 1 + 1.1 * fgmask_display)
    
    field = c_decay * field + c_emit * fgmask_display
    field = np.minimum(1, field)
    
    fx = visualize(field, fgmask_display, count_on)
    
    cv.imshow('frame', frame)
    cv.imshow('mask', fgmask)
    cv.imshow
    cv.imshow('fx', fx)
    if cv.waitKey(1) & 0xFF == ord('q'):
        break
    fps.update()
    
    t += 0.06667

fps.stop()
cap.release()
cv.destroyAllWindows()

---

### Prerecorded video

In [ ]:
def background_subtraction_mosaic(
    video_file,
    save_file,
    subtractors,
    median_blur_size,
    gaussian_blur_size,
    scale,
    spacing,
    show_current_frame=False):
    """Run a background subtraction configuration on a source
    video file. Save the output to another video.
    """
    # Create a VideoCapture from a video file
    cap = cv.VideoCapture(video_file)
    # Get video file attributes for creating the output video
    n_frames = int(cap.get(cv.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv.CAP_PROP_FPS))
    
    # Codec for the VideoWriter
    fourcc = cv.VideoWriter_fourcc(*'X264')
    # Create a VideoWriter
    w_each = int(scale * width)
    h_each = int(scale * height)
    ncols = len(subtractors) + 1
    nrows = max([len(s) for s in subtractors])
    w = w_each * ncols + spacing * (ncols - 1)
    h = h_each * nrows + spacing * (nrows - 1)
    vid_out_size = (w, h)
    vid_out = cv.VideoWriter(
        save_file, 
        fourcc, 
        fps, 
        vid_out_size,
        True)
    
    # background is off-white
    frame_out = 235 * np.ones((h, w, 3)).astype(np.uint8)
    for _ in range(n_frames):
        ret, frame = cap.read()
        if show_current_frame:
            cv.imshow('frame', frame)
            cv.waitKey(1)
        fresized = imutils.resize(frame, height=h_each, width=w_each)
        frame_out[:h_each, :w_each, :] = fresized
        for x, col in enumerate(subtractors):
            x0 = (x + 1) * (w_each + spacing) 
            x1 = x0 + w_each
            for y, subtractor in enumerate(col):
                y0 = y * (h_each + spacing)
                y1 = y0 + h_each
                mask = subtractor.apply(frame)
                if median_blur_size > 0:
                    mask = cv.medianBlur(mask, median_blur_size)
                if gaussian_blur_size > 0:
                    shape = (gaussian_blur_size, gaussian_blur_size)
                    mask = cv.GaussianBlur(mask, shape, 0)
                
                mresized = imutils.resize(
                    mask, 
                    height=h_each,
                    width=w_each)
                frame_out[y0:y1, x0:x1, :] = np.stack(
                    [mresized]*3,
                    axis=-1)
        vid_out.write(frame_out)
        
    cap.release()
    vid_out.release()
    cv.destroyAllWindows()
    pass

#### Define some subtraction algorithm configs to test

In [ ]:
save_dir = 'masks'
os.makedirs(save_dir, exist_ok=True)

median_value = 11
gaussian_value = 5

subtractors = [
    [
        cv.createBackgroundSubtractorKNN(500, 400, True),
        cv.createBackgroundSubtractorKNN(200, 400, True),
        cv.createBackgroundSubtractorKNN(200, 1000, True),
        cv.createBackgroundSubtractorKNN(500, 200, True),
        cv.createBackgroundSubtractorKNN(500, 1000, True),
        cv.createBackgroundSubtractorKNN(500, 400, False)
    ],
    
    [
        cv.createBackgroundSubtractorMOG2(500, 16, True),
        cv.createBackgroundSubtractorMOG2(200, 16, True),
        cv.createBackgroundSubtractorMOG2(200, 32, True),
        cv.createBackgroundSubtractorMOG2(500, 10, True),
        cv.createBackgroundSubtractorMOG2(500, 32, True),
        cv.createBackgroundSubtractorMOG2(500, 16, False)
    ],
    
    [
        cv.bgsegm.createBackgroundSubtractorCNT(15, True, 900),
        cv.bgsegm.createBackgroundSubtractorCNT(15, True, 400),
        cv.bgsegm.createBackgroundSubtractorCNT(5, True, 900),
        cv.bgsegm.createBackgroundSubtractorCNT(45, True, 900),
        cv.bgsegm.createBackgroundSubtractorCNT(5, True, 400),
        cv.bgsegm.createBackgroundSubtractorCNT(45, True, 400)
    ],
    
    [
        cv.bgsegm.createBackgroundSubtractorGMG(45, 0.8),
        cv.bgsegm.createBackgroundSubtractorGMG(45, 0.9),
        cv.bgsegm.createBackgroundSubtractorGMG(45, 0.95),
        cv.bgsegm.createBackgroundSubtractorGMG(45, 0.7),
        cv.bgsegm.createBackgroundSubtractorGMG(45, 0.6),
        cv.bgsegm.createBackgroundSubtractorGMG(20, 0.8)
    ],
    
    [
        cv.bgsegm.createBackgroundSubtractorGSOC(replaceRate=0.003, propagationRate=0.01),
        cv.bgsegm.createBackgroundSubtractorGSOC(replaceRate=0.003, propagationRate=0.025),
        cv.bgsegm.createBackgroundSubtractorGSOC(replaceRate=0.006, propagationRate=0.01),
        cv.bgsegm.createBackgroundSubtractorGSOC(replaceRate=0.0015, propagationRate=0.01),
        cv.bgsegm.createBackgroundSubtractorGSOC(replaceRate=0.006, propagationRate=0.025),
        cv.bgsegm.createBackgroundSubtractorGSOC(replaceRate=0.0015, propagationRate=0.025)
    ],
    
    [
        cv.bgsegm.createBackgroundSubtractorLSBP(noiseRemovalThresholdFacBG=0.0004, noiseRemovalThresholdFacFG=0.0008),
        cv.bgsegm.createBackgroundSubtractorLSBP(noiseRemovalThresholdFacBG=0.0004, noiseRemovalThresholdFacFG=0.00016),
        cv.bgsegm.createBackgroundSubtractorLSBP(noiseRemovalThresholdFacBG=0.0002, noiseRemovalThresholdFacFG=0.0004),
        cv.bgsegm.createBackgroundSubtractorLSBP(noiseRemovalThresholdFacBG=0.0008, noiseRemovalThresholdFacFG=0.0004),
        cv.bgsegm.createBackgroundSubtractorLSBP(noiseRemovalThresholdFacBG=0.0002, noiseRemovalThresholdFacFG=0.0016),
        cv.bgsegm.createBackgroundSubtractorLSBP(noiseRemovalThresholdFacBG=0.0008, noiseRemovalThresholdFacFG=0.0016)
    ],
    
    [
        cv.bgsegm.createBackgroundSubtractorMOG(200, 5, 0.7),
        cv.bgsegm.createBackgroundSubtractorMOG(200, 3, 0.7),
        cv.bgsegm.createBackgroundSubtractorMOG(200, 7, 0.7),
        cv.bgsegm.createBackgroundSubtractorMOG(200, 5, 0.55),
        cv.bgsegm.createBackgroundSubtractorMOG(200, 5, 0.85),
        cv.bgsegm.createBackgroundSubtractorMOG(200, 7, 0.85)
    ]
]

background_subtraction_mosaic(
    'test.mp4',
    os.path.join(save_dir, 'mosaic1013.mp4'),
    subtractors,
    median_value,
    gaussian_value,
    0.37,
    2,
    True)